In [ ]:
import tensorflow as tfimport numpy as npimport mathfrom sklearn import datasetsfrom sklearn.model_selection import train_test_splitfrom tensorflow.python.framework import opsops.reset_default_graph()boston = datasets.load_boston()data = boston["data"]target = boston["target"]N_INSTANCES = data.shape[0]N_INPUT = data.shape[1]N_CLASSES = 3TEST_SIZE = 0.1TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))batch_size = 40training_epochs = 15learning_rate = 0.005display_step = 1hidden_size = 256target_ = np.zeros((N_INSTANCES, N_CLASSES))data_train, data_test, target_train, target_test = train_test_split(data, target_, test_size=0.1, random_state=100)#TODO: figure out the right placeholders for x_data and y_targetx_data = tf.placeholder(shape=[None, N_INPUT], dtype=tf.float32)y_target = tf.placeholder(shape=[None, N_CLASSES], dtype=tf.float32)# creates activation functiondef gaussian_function(input_layer):initial = math.exp(-2*math.pow(input_layer, 2))return initialnp_gaussian_function = np.vectorize(gaussian_function)def d_gaussian_function(input_layer):initial = -4 * input_layer * math.exp(-2*math.pow(input_layer, 2))return initialnp_d_gaussian_function = np.vectorize(d_gaussian_function)np_d_gaussian_function_32 = lambda input_layer: np_d_gaussian_function(input_layer).astype(np.float32)def tf_d_gaussian_function(input_layer, name=None):with ops.name_scope(name, "d_gaussian_function", [input_layer]) as name:y = tf.py_func(np_d_gaussian_function_32, [input_layer], [tf.float32], name=name, stateful=False)return y[0]def py_func(func, inp, Tout, stateful=True, name=None, grad=None):rnd_name = 'PyFunGrad' + str(np.random.randint(0, 1E+8))tf.RegisterGradient(rnd_name)(grad)g = tf.get_default_graph()with g.gradient_override_map({"PyFunc": rnd_name}):return tf.py_func(func, inp, Tout, stateful=stateful, name=name)def gaussian_function_grad(op, grad):input_variable = op.inputs[0]n_gr = tf_d_gaussian_function(input_variable)return grad * n_grnp_gaussian_function_32 = lambda input_layer: np_gaussian_function(input_layer).astype(np.float32)def tf_gaussian_function(input_layer, name=None):with ops.name_scope(name, "gaussian_function", [input_layer]) as name:y = py_func(np_gaussian_function_32, [input_layer], [tf.float32],name=name, grad=gaussian_function_grad)return y[0]# end of defining activation functiondef rbf_network(input_layer, weights, biases):layer1 = tf.add(tf.matmul(tf_gaussian_function(input_layer), weights['h1']), biases['b1'])layer2 = tf.add(tf.matmul(tf_gaussian_function(layer1), weights['h2']), biases['b2'])#layer3 = tf.add(tf.matmul(tf_gaussian_function(layer2), weights['h3']), biases['b3'])output = tf.matmul(tf_gaussian_function(layer2), weights['output']) + biases['output']return outputweights = {'h1': tf.Variable(tf.random_normal([N_INPUT, hidden_size], stddev=0.1)),'h2': tf.Variable(tf.random_normal([hidden_size, hidden_size], stddev=0.1)),#'h3': tf.Variable(tf.random_normal([hidden_size, hidden_size], stddev=0.1)),'output': tf.Variable(tf.random_normal([hidden_size, N_CLASSES], stddev=0.1))}biases = {'b1': tf.Variable(tf.random_normal([hidden_size])),'b2': tf.Variable(tf.random_normal([hidden_size])),#'b3': tf.Variable(tf.random_normal([hidden_size])),'output': tf.Variable(tf.random_normal([N_CLASSES]))}pred = rbf_network(x_data, weights, biases)cost = tf.reduce_mean(tf.squared_difference(pred, y_target))my_opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_target, 1))accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))init = tf.global_variables_initializer()sess = tf.InteractiveSession()sess.run(init)# Training loopfor epoch in range(training_epochs):avg_cost = 0.total_batch = int(data_train.shape[0] / batch_size)for i in range(total_batch):randidx = np.random.randint(int(TRAIN_SIZE), size=batch_size)batch_xs = data_train[randidx, :]batch_ys = target_train[randidx, :]sess.run(my_opt, feed_dict={x_data: batch_xs, y_target: batch_ys})avg_cost += sess.run(cost, feed_dict={x_data: batch_xs, y_target: batch_ys})/total_batchif epoch % display_step == 0:print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))train_accuracy = sess.run(accuracy, feed_dict={x_data: batch_xs, y_target: batch_ys})print("Training accuracy: %.3f" % train_accuracy)test_acc = sess.run(accuracy, feed_dict={x_data: data_test, y_target: target_test})print("Test accuracy: %.3f" % test_acc)sess.close()